In [ ]:
# codigo de Mario
# string_columns = [column for column in df_amazon.columns if df_amazon[column].dtype == 'object']
# df_amazon[string_columns] = df_amazon.loc[:,string_columns].apply(lambda x: x.str.lower())
# codigo de David
# df_amazon = df_amazon.apply(lambda x: x.str.lower() if x.dtype == 'object' else x)

In [ ]:
import pandas as pd

In [ ]:
def transformacion (df,letras):
    df.rename(columns={'show_id':'id'},inplace= True) # renombro la columna "show_id" a "id"
    df['id'] = [letras + elemento for elemento in df['id']] # agrego la letra correspodiente
    df.set_index('id',inplace=True) # asigno a la columna "id" como indice
    df['rating'].fillna('G',inplace=True) # relleno los valores NaN de la columna "rating" con "G"
    df['date_added']= pd.to_datetime(df['date_added']) #cambio el formato de las fechas
    string_columns = [column for column in df.columns if df[column].dtype == 'object'] #elijo cual de las columnas es de tipo 'object'
    df[string_columns] = df.loc[:,string_columns].apply(lambda x: x.str.lower()) # transformo en minusculas a las columnas de tipo object
    df['duration'].fillna ('NaN NaN', inplace=True) #reemplazo el los valores faltantes de la columna duration por "NaN NaN" con el fin de poder dividirla depues
    duracion = df['duration'].str.split() #separo la columna "duration" en 2
    duration_int = [i[0] for i in duracion] # asigno los primeros valores a "duration_int"
    duration_type= [i[1] for i in duracion] # asigno los primeros valores a "duration_type"
    df.insert(9,'duration_int',duration_int) # inserto la columna "duration_int"
    df.insert(10,'duration_type',duration_type) # inserto la columna "duration_type"
    df.drop(columns= 'duration', inplace= True) # elimino la columna duration
    df['duration_type'].replace('seasons','season', inplace=True) # corrijo los valores "seasons" por "season" para tener un solo valor y facilite la busqueda
    return df

In [ ]:
# Importación de archivos
df_disney = pd.read_csv (r'Datasets/disney_plus_titles-score.csv', encoding= 'UTF-8')
df_amazon = pd.read_csv (r'Datasets/amazon_prime_titles-score.csv', encoding= 'UTF-8')
df_hulu = pd.read_csv (r'Datasets/hulu_titles-score (2).csv', encoding= 'UTF-8')
df_netflix = pd.read_csv (r'Datasets/netflix_titles-score.csv', encoding= 'UTF-8')

In [ ]:
# Aplicación de cambios
df_amazon = transformacion (df_amazon,'a')
df_disney = transformacion (df_disney,'d')
df_hulu = transformacion (df_hulu,'h')
df_netflix = transformacion (df_netflix,'n')
# Dataframe final para consultas
df_final = pd.concat([df_amazon,df_disney,df_hulu,df_netflix],axis=1)

In [16]:
df_final.to_csv('df_final.to_csv')

In [ ]:
# 1) Cantidad de veces que aparece una keyword en el título de peliculas/series, por plataforma
def cantidad_keyword (data_frame, keyword):
    contador = 0
    lista = []
    for palabra in data_frame.title:
        palabra = palabra.split()
        for elemento in palabra:
            lista.append(elemento)
    contador = lista.count(keyword)
    return contador

# 2) Cantidad de películas por plataforma con un puntaje mayor a XX en determinado año
def cantidad_peliculas_puntaje (data_frame, anio, puntaje):
    return len(data_frame[(data_frame['date_added'].dt.year== anio) & (data_frame['score'] >= puntaje)].value_counts())

# 3)) Segunda película con mayor score para una plataforma determinada, según el orden alfabético de los títulos
def segundo_maximo (data_frame):
    return data_frame[data_frame['score']==data_frame['score'].max()].sort_values(by=['title']).iloc[1]

# 4) Película que más duró según año, plataforma y tipo de duración
def mas_duracion (data_frame, tipo):
    return data_frame[data_frame['duration_type']== tipo].sort_values(by= ['duration_int']).iloc[-1]

# 5) Cantidad de series y películas por rating
def cantidad_series_peliculas (data_frame):
    return data_frame['rating'].value_counts()

In [ ]:
cantidad_keyword (df_amazon, 'johnson')
cantidad_peliculas_puntaje (df_amazon, 2021, 30)
segundo_maximo (df_amazon)
mas_duracion (df_amazon,'season')
cantidad_series_peliculas (df_amazon) 